In [1]:
#데이터 핸들링 라이브러리
import pandas as pd
import numpy as np
#데이터 시각화 라이브러리
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcParams.update({'font.family':'AppleGothic'})
mpl.rc('axes', unicode_minus=False)
%config InlineBackend.figure_format = 'retina'

## 데이터 전처리 계획
1. 데이터 불러오기, 결측치 처리하기
2. 상대적으로 중요성이 떨어져보이는 피처 삭제, 필요하면 피처명을 직관적으로 다시 네이밍하기
3. 비숫자-문자열 독립변수 인코딩하기
4. 데이터 정규화
5. 훈련-테스트셋 분리

## 모델 학습 계획
1. 모델 생성
2. Cross_validation을 통한 모델학습
3. Accuracy, Precision,Recall, ROC 4개의 지표로 분류모델의 정확도 평가하기

In [2]:
#1. 데이터 불러오기 from TAAS: http://taas.koroad.or.kr/web/shp/sbm/initGisAnals.do?menuId=WEB_KMP_GIS_TAS
## 해당 경로에 있는 .csv 파일명 리스트 가져오기

import os
import time
t_0= time.time()

path = './교통사고_2021/'
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.xls')] ## 파일명 끝이 .xls인 경우

## csv 파일들을 DataFrame으로 불러와서 concat

death_df = pd.DataFrame()
for i in file_list_py:
    data = pd.read_html(path + i)
    data= data[0]
    death_df = pd.concat([death_df,data])

death_df = death_df.reset_index(drop = True)
print(f'데이터 불러오기 및 통합에 걸린 시간은 {str(time.time()-t_0)} 입니다')
death_df

데이터 불러오기 및 통합에 걸린 시간은 30.65006113052368 입니다


,사고번호,사고일시,요일,시군구,사고내용,사망자수,중상자수,경상자수,부상신고자수,사고유형,...,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도
0,2021010100100022,2021년 1월 1일 02시,금요일,강원도 양양군 현남면,중상사고,0,1,1,0,차량단독 - 공작물충돌,...,맑음,단일로 - 기타,승용,여,19세,상해없음,NaN,NaN,NaN,NaN
1,2021010100100029,2021년 1월 1일 04시,금요일,강원도 횡성군 둔내면,경상사고,0,0,2,0,차량단독 - 공작물충돌,...,맑음,교차로 - 교차로안,승용,남,21세,경상,NaN,NaN,NaN,NaN
2,2021010100100040,2021년 1월 1일 08시,금요일,강원도 강릉시 강문동,중상사고,0,1,0,0,차대사람 - 횡단중,...,맑음,단일로 - 기타,화물,남,53세,상해없음,보행자,여,20세,중상
3,2021010100100065,2021년 1월 1일 11시,금요일,강원도 강릉시 옥계면,경상사고,0,0,4,0,차대차 - 측면충돌,...,맑음,교차로 - 교차로안,승용,남,43세,상해없음,화물,남,49세,경상
4,2021010100100089,2021년 1월 1일 13시,금요일,강원도 동해시 천곡동,경상사고,0,0,2,0,차대차 - 측면충돌,...,맑음,단일로 - 기타,승용,남,62세,상해없음,승용,남,24세,경상
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194089,2021123100100335,2021년 12월 31일 16시,금요일,충청남도 천안시 동남구 광덕면,경상사고,0,0,3,0,차대차 - 측면충돌,...,맑음,교차로 - 교차로안,승용,여,32세,경상,승용,남,42세,경상
194090,2021123100100383,2021년 12월 31일 17시,금요일,충청남도 부여군 장암면,중상사고,0,1,0,0,차대차 - 추돌,...,맑음,단일로 - 기타,승용,남,24세,상해없음,자전거,남,78세,중상
194091,2021123100100384,2021년 12월 31일 17시,금요일,충청남도 천안시 동남구 청당동,경상사고,0,0,3,0,차대차 - 측면충돌,...,맑음,단일로 - 기타,승용,남,25세,경상,승용,남,22세,경상
194092,2021123100100578,2021년 12월 31일 21시,금요일,충청남도 보령시 명천동,경상사고,0,0,1,0,차대차 - 추돌,...,맑음,기타 - 기타,승용,남,59세,상해없음,승용,여,28세,경상


In [3]:
"""
결측치 확인하기 => 피해운전자의 차종,성별,연령,상해정도를 파악하지 못하는 데이터가 6687건이 존재함
원본데이터 194094에 비하면 삭제해도 학습에 영향력을 크게 미치지 못하뿐더러, 이러한 데이터들이
특정 편향성을 갖고 존재할 것이라 보기 어렵다(아마도 기록 누락에 의해 이러한 데이터가 존재할 것이기 때문)
그렇기 때문에 데이터를 제거해도 될것이라 판단하였음
"""
death_df.isnull().sum()
death_df.dropna(inplace=True)
death_df.isnull().sum()

사고번호          0
사고일시          0
요일            0
시군구           0
사고내용          0
사망자수          0
중상자수          0
경상자수          0
부상신고자수        0
사고유형          0
법규위반          0
노면상태          0
기상상태          0
도로형태          0
가해운전자 차종      0
가해운전자 성별      0
가해운전자 연령      0
가해운전자 상해정도    0
피해운전자 차종      0
피해운전자 성별      0
피해운전자 연령      0
피해운전자 상해정도    0
dtype: int64

In [5]:
#데이터 정보 찾기
print(death_df.info())
death_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 187407 entries, 2 to 194093
Data columns (total 22 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   사고번호        187407 non-null  int64 
 1   사고일시        187407 non-null  object
 2   요일          187407 non-null  object
 3   시군구         187407 non-null  object
 4   사고내용        187407 non-null  object
 5   사망자수        187407 non-null  int64 
 6   중상자수        187407 non-null  int64 
 7   경상자수        187407 non-null  int64 
 8   부상신고자수      187407 non-null  int64 
 9   사고유형        187407 non-null  object
 10  법규위반        187407 non-null  object
 11  노면상태        187407 non-null  object
 12  기상상태        187407 non-null  object
 13  도로형태        187407 non-null  object
 14  가해운전자 차종    187407 non-null  object
 15  가해운전자 성별    187407 non-null  object
 16  가해운전자 연령    187407 non-null  object
 17  가해운전자 상해정도  187407 non-null  object
 18  피해운전자 차종    187407 non-null  object
 19  피해운전자 성별    187407 non-

,사고번호,사고일시,요일,시군구,사고내용,사망자수,중상자수,경상자수,부상신고자수,사고유형,...,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도
2,2021010100100040,2021년 1월 1일 08시,금요일,강원도 강릉시 강문동,중상사고,0,1,0,0,차대사람 - 횡단중,...,맑음,단일로 - 기타,화물,남,53세,상해없음,보행자,여,20세,중상
3,2021010100100065,2021년 1월 1일 11시,금요일,강원도 강릉시 옥계면,경상사고,0,0,4,0,차대차 - 측면충돌,...,맑음,교차로 - 교차로안,승용,남,43세,상해없음,화물,남,49세,경상
4,2021010100100089,2021년 1월 1일 13시,금요일,강원도 동해시 천곡동,경상사고,0,0,2,0,차대차 - 측면충돌,...,맑음,단일로 - 기타,승용,남,62세,상해없음,승용,남,24세,경상
5,2021010100100123,2021년 1월 1일 15시,금요일,강원도 강릉시 포남동,사망사고,1,0,0,0,차대차 - 측면충돌,...,맑음,교차로 - 교차로안,원동기,남,16세,상해없음,승용,여,50세,상해없음
6,2021010100100146,2021년 1월 1일 17시,금요일,강원도 강릉시 옥천동,경상사고,0,0,2,0,차대차 - 추돌,...,맑음,교차로 - 교차로부근,승용,여,33세,상해없음,승용,남,27세,경상


In [115]:
#2.상대적으로 덜 중요해 보이는 피처 삭제하기

'''
1. 2021년도 전국교통사고통계에서 '당사자종별_1당_대분류','당사자종별_2당_대분류'에 해당하는 위치에
'가해자_당사자종별', '피해자_당사자종별'이 있었다는 점과
2. death_df.groupby('당사자종별_2당_대분류').describe()에서만 보행자가 카테고리로 존재하는것으로 비추어보아
'당사자종별_1당_대분류','당사자종별_2당_대분류' = '가해자_당사자종별', '피해자_당사자종별' 이라고 판단하였음
'''

death_df.drop(['요일','발생위치X_UTMK','발생위치Y_UTMK'],axis=1,inplace=True)
death_df.groupby('당사자종별_2당_대분류').describe()
death_df.rename({'당사자종별_1당_대분류':'가해자_당사자종별','당사자종별_2당_대분류':'피해자_당사자종별'},axis=1,inplace=True)


In [116]:
#시간에 따른 시계가 교통사고의 요인중 하나일 수 있으므로 발생년월일시에서 시간만 떼어 새로운 컬럼으로 만듬
time_lst= list(death_df['발생년월일시'])
hour_lst=[]
for time in time_lst:
    hour_lst.append(int(str(time)[-2:]))

death_df['발생시각']= hour_lst
death_df

,발생년월일시,주야,요일,사망자수,사상자수,중상자수,경상자수,부상신고자수,발생지시도,발생지시군구,...,사고유형_중분류,사고유형,법규위반,도로형태_대분류,도로형태,가해자_당사자종별,피해자_당사자종별,경도,위도,발생시각
0,2012010101,야간,일,1,1,0,0,0,서울,은평구,...,차도통행중,차도통행중,안전운전 의무 불이행,단일로,기타단일로,승용차,보행자,126.931890,37.612680,1
1,2012010101,야간,일,1,6,5,0,0,전북,정읍시,...,정면충돌,정면충돌,중앙선 침범,단일로,기타단일로,승용차,승용차,126.909523,35.633956,1
2,2012010108,주간,일,1,1,0,0,0,충남,청양군,...,공작물충돌,공작물충돌,안전운전 의무 불이행,단일로,기타단일로,승용차,없음,126.830281,36.491268,8
3,2012010110,주간,일,2,2,0,0,0,경남,합천군,...,측면충돌,측면충돌,과속,교차로,교차로내,승합차,승용차,128.155984,35.733503,10
4,2012010103,야간,일,1,1,0,0,0,경북,예천군,...,도로이탈,도로이탈 추락,안전운전 의무 불이행,단일로,기타단일로,승용차,없음,128.284180,36.506769,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39939,2021112608,주간,금,1,1,0,0,0,경기,평택시,...,기타,기타,안전운전 의무 불이행,교차로,교차로내,승용차,보행자,127.057882,37.081528,8
39940,2021112717,주간,토,1,1,0,0,0,경기,평택시,...,기타,기타,안전운전 의무 불이행,단일로,기타단일로,승용차,자전거,127.136562,36.998521,17
39941,2021120711,주간,화,1,1,0,0,0,경기,평택시,...,추돌,추돌,안전운전 의무 불이행,기타,기타,화물차,화물차,126.926540,36.963284,11
39942,2021123121,야간,금,1,1,0,0,0,강원,양구군,...,전복,전복,중앙선 침범,단일로,기타단일로,승용차,없음,127.985386,38.097913,21


In [131]:
#무엇을 예측할 것인가?
'''
이번 분석에서 예측하고자 하는것은 사상자의 수가 아니다. 피해자와 가해자가 얼마나 탑승하였는지는 몇몇 피처들과의 상관관계가
있을 수 있으나 강력할 것이라 기대하기는 어렵다. 그보다는 교통사고의 심각한 정도를 확인하고자 한다.
그에 대한 기준으로 사망자수,중상자수,경상자수 컬럼을 활용한다. 새로운 '심각도' 컬럼,y로 0:경상자_존재,1:중상자_존재,2:사상자_존재로 한다
사망자,중상자,경상자가 존재할 경우 2번에, 중상자,경상자가 존재할 경우 1번에 분류하도록 한다
'''
casualties= death_df[['사망자수','중상자수','경상자수']]
for i in range(len(casualties)):
    if casualties['사망자수'][i] != 0:
        death_df['심각도']=2
    elif casualties['중상자수'][i] !=0:
        death_df['심각도']=1
    else:
        death_df['심각도']=0

death_df


,발생년월일시,주야,요일,사망자수,사상자수,중상자수,경상자수,부상신고자수,발생지시도,발생지시군구,...,사고유형,법규위반,도로형태_대분류,도로형태,가해자_당사자종별,피해자_당사자종별,경도,위도,발생시각,심각도
0,2012010101,야간,4,1,1,0,0,0,서울,은평구,...,16,6,단일로,7,6,4,126.931890,37.612680,1,2
1,2012010101,야간,4,1,6,5,0,0,전북,정읍시,...,10,7,단일로,7,6,7,126.909523,35.633956,1,2
2,2012010108,주간,4,1,1,0,0,0,충남,청양군,...,1,6,단일로,7,6,9,126.830281,36.491268,8,2
3,2012010110,주간,4,2,2,0,0,0,경남,합천군,...,19,0,교차로,2,7,7,128.155984,35.733503,10,2
4,2012010103,야간,4,1,1,0,0,0,경북,예천군,...,5,6,단일로,7,6,9,128.284180,36.506769,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39939,2021112608,주간,0,1,1,0,0,0,경기,평택시,...,2,6,교차로,2,6,4,127.057882,37.081528,8,2
39940,2021112717,주간,5,1,1,0,0,0,경기,평택시,...,2,6,단일로,7,6,13,127.136562,36.998521,17,2
39941,2021120711,주간,6,1,1,0,0,0,경기,평택시,...,18,6,기타,5,12,15,126.926540,36.963284,11,2
39942,2021123121,야간,0,1,1,0,0,0,강원,양구군,...,9,7,단일로,7,6,9,127.985386,38.097913,21,2


In [117]:
#우선 의사결정나무와 랜덤포레스트 머신러닝 기법을 활용해보도록 한다. 그러므로 범주형 변수의 인코딩으로 레이블인코딩을 해도 되게 된다
#3.범주형 변수 인코딩하기
#한번만 인코딩해야됨. 두번하면 안됨
from sklearn.preprocessing import LabelEncoder

categories_backup=[]
features=['요일','사고유형','법규위반','도로형태','가해자_당사자종별','피해자_당사자종별']
for feature in features:
    LE= LabelEncoder()
    LE= LE.fit( death_df[feature])
    #카테고리 백업해두기
    categories_backup.append(list(LE.classes_))
    #인코딩하기
    death_df[feature]= LE.transform(death_df[feature])

In [118]:
#백업해둔 카테고리명 다시 한번 확인하기
categories_backup

[['금', '목', '수', '월', '일', '토', '화'],
 ['경보기 무시',
  '공작물충돌',
  '기타',
  '길가장자리구역통행중',
  '도로이탈 기타',
  '도로이탈 추락',
  '보도통행중',
  '전도',
  '전도전복',
  '전복',
  '정면충돌',
  '주/정차차량 충돌',
  '주정차중 추돌',
  '직전진행',
  '진행중 추돌',
  '차단기돌파',
  '차도통행중',
  '철길건널목',
  '추돌',
  '측면충돌',
  '횡단중',
  '후진중충돌'],
 ['과속',
  '교차로 통행방법 위반',
  '기타',
  '보행자 보호의무 위반',
  '신호위반',
  '안전거리 미확보',
  '안전운전 의무 불이행',
  '중앙선 침범'],
 ['고가도로위',
  '교량위',
  '교차로내',
  '교차로부근',
  '교차로횡단보도내',
  '기타',
  '기타/불명',
  '기타단일로',
  '불명',
  '주차장',
  '지하차도(도로)내',
  '철길건널목',
  '터널안',
  '횡단보도부근',
  '횡단보도상'],
 ['개인형이동수단(PM)',
  '건설기계',
  '기타',
  '농기계',
  '불명',
  '사륜오토바이(ATV)',
  '승용차',
  '승합차',
  '원동기장치자전거',
  '이륜차',
  '자전거',
  '특수차',
  '화물차'],
 ['개인형이동수단(PM)',
  '건설기계',
  '기타',
  '농기계',
  '보행자',
  '불명',
  '사륜오토바이(ATV)',
  '승용차',
  '승합차',
  '없음',
  '열차',
  '원동기장치자전거',
  '이륜차',
  '자전거',
  '특수차',
  '화물차']]

In [120]:
X=death_df[['발생시각','사고유형','법규위반','도로형태','가해자_당사자종별','피해자_당사자종별']]
y_casualty=death_df['사상자수']

In [121]:
X

,발생시각,사고유형,법규위반,도로형태,가해자_당사자종별,피해자_당사자종별
0,1,16,6,7,6,4
1,1,10,7,7,6,7
2,8,1,6,7,6,9
3,10,19,0,2,7,7
4,3,5,6,7,6,9
...,...,...,...,...,...,...
39939,8,2,6,2,6,4
39940,17,2,6,7,6,13
39941,11,18,6,5,12,15
39942,21,9,7,7,6,9
